# Chapter 11: Union and Intersection Types

---

## 11.1 Union Types

Union types allow you to declare that a value can be one of several types, providing flexibility while maintaining type safety. This is one of TypeScript's most powerful features for handling values that could have different shapes or types at runtime.

### 11.1.1 Defining Union Types

**Basic Union Syntax:**

```typescript
// A string or a number
type StringOrNumber = string | number;

// Usage in variables
let value: StringOrNumber;

value = "hello";  // ✅ OK - string
value = 42;       // ✅ OK - number
// value = true;  // ❌ Error: Type 'boolean' is not assignable to type 'string | number'

// Union in function parameters
function formatInput(input: string | number): string {
  if (typeof input === "string") {
    return input.toUpperCase();
  }
  return input.toFixed(2);
}

console.log(formatInput("hello"));  // "HELLO"
console.log(formatInput(3.14159));  // "3.14"

// Union in arrays
const mixedArray: (string | number)[] = ["one", 2, "three", 4];
// Can also write: Array<string | number>

// Union with objects
type User = { name: string; age: number };
type Admin = { name: string; permissions: string[] };
type Person = User | Admin;

const user: Person = { name: "John", age: 30 };
const admin: Person = { name: "Jane", permissions: ["read", "write"] };
```

**Union Type Syntax:**

```
┌─────────────────────────────────────────────────────────────────────┐
│                      Union Type Syntax                               │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│   Type1 | Type2 | Type3 | ...                                       │
│                                                                     │
│   Examples:                                                         │
│                                                                     │
│   ┌──────────────────────────────────────────────────────────────┐ │
│   │ // Simple unions                                              │ │
│   │ type Status = "active" | "inactive" | "pending";              │ │
│   │ type ID = string | number;                                     │ │
│   │                                                                │ │
│   │ // Complex unions                                             │ │
│   │ type Response = Success | Error | Loading;                    │ │
│   │                                                                │ │
│   │ // Nullable unions                                            │ │
│   │ type MaybeString = string | null | undefined;                 │ │
│   │                                                                │ │
│   │ // Function parameter unions                                  │ │
│   │ function process(value: string | number | boolean): void { }    │ │
│   └──────────────────────────────────────────────────────────────┘ │
│                                                                     │
│   Important: Union types are not intersection types!                │
│   Union = "one of these" (OR)                                       │
│   Intersection = "all of these" (AND)                               │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

### 11.1.2 Union Type Syntax

**Working with Union Types:**

```typescript
// Union with type aliases
type HttpMethod = "GET" | "POST" | "PUT" | "DELETE" | "PATCH";
type HttpStatus = 200 | 201 | 204 | 400 | 401 | 403 | 404 | 500;

function makeRequest(url: string, method: HttpMethod): void {
  console.log(`${method} ${url}`);
}

makeRequest("/api/users", "GET");     // ✅ OK
makeRequest("/api/users", "POST");    // ✅ OK
// makeRequest("/api/users", "get");  // ❌ Error: "get" is not assignable

// Union with objects - discriminated unions (preview of 11.3)
type Shape = 
  | { kind: "circle"; radius: number }
  | { kind: "rectangle"; width: number; height: number }
  | { kind: "square"; side: number };

function getArea(shape: Shape): number {
  switch (shape.kind) {
    case "circle":
      return Math.PI * shape.radius ** 2;
    case "rectangle":
      return shape.width * shape.height;
    case "square":
      return shape.side ** 2;
  }
}

// Union with null/undefined (nullable types)
type Maybe<T> = T | null | undefined;

function findUser(id: number): Maybe<{ id: number; name: string }> {
  if (id <= 0) return null;
  return { id, name: "User " + id };
}

const user = findUser(1);
// Must check for null before using
if (user) {
  console.log(user.name);
}
```

### 11.1.3 Working with Union Values

**Type Narrowing Basics:**

```typescript
// Type narrowing with typeof
function processValue(value: string | number | boolean): void {
  // Inside here, value is string | number | boolean
  
  if (typeof value === "string") {
    // TypeScript knows value is string here
    console.log(value.toUpperCase());
    console.log(value.length);
  } else if (typeof value === "number") {
    // TypeScript knows value is number here
    console.log(value.toFixed(2));
    console.log(value * 2);
  } else {
    // TypeScript knows value is boolean here (exhausted other options)
    console.log(value ? "Yes" : "No");
  }
}

// Type narrowing with instanceof
class Cat {
  meow(): void {
    console.log("Meow");
  }
}

class Dog {
  bark(): void {
    console.log("Woof");
  }
}

function makeSound(animal: Cat | Dog): void {
  if (animal instanceof Cat) {
    // TypeScript knows animal is Cat
    animal.meow();
  } else {
    // TypeScript knows animal is Dog
    animal.bark();
  }
}

// Type narrowing with in operator
interface Car {
  drive(): void;
  wheels: number;
}

interface Boat {
  sail(): void;
  draft: number;
}

function operate(vehicle: Car | Boat): void {
  if ("drive" in vehicle) {
    // TypeScript knows vehicle is Car
    vehicle.drive();
    console.log(`Has ${vehicle.wheels} wheels`);
  } else {
    // TypeScript knows vehicle is Boat
    vehicle.sail();
    console.log(`Draft: ${vehicle.draft}`);
  }
}
```

---

## 11.2 Type Narrowing

Type narrowing is the process of refining a union type to a more specific type based on runtime checks. This is essential for working safely with union types.

### 11.2.1 `typeof` Type Guards

The `typeof` operator can be used to narrow types to primitive types: `string`, `number`, `bigint`, `boolean`, `symbol`, `undefined`, `object`, or `function`.

**Using `typeof` for Type Narrowing:**

```typescript
// Basic typeof narrowing
function formatValue(value: string | number | boolean): string {
  if (typeof value === "string") {
    // TypeScript knows: value is string
    return value.trim().toUpperCase();
  }
  
  if (typeof value === "number") {
    // TypeScript knows: value is number
    if (isNaN(value)) {
      return "Invalid number";
    }
    return value.toLocaleString("en-US", {
      minimumFractionDigits: 2,
      maximumFractionDigits: 2
    });
  }
  
  // TypeScript knows: value is boolean (exhausted other options)
  return value ? "Yes" : "No";
}

// Usage
console.log(formatValue("  hello world  ")); // "HELLO WORLD"
console.log(formatValue(1234.5));             // "1,234.50"
console.log(formatValue(true));               // "Yes"
console.log(formatValue(false));              // "No"

// typeof with objects and null (quirk)
function checkNull(value: object | null): void {
  if (typeof value === "object") {
    // TypeScript knows: value is object | null
    // Because typeof null === "object" in JavaScript!
    if (value === null) {
      console.log("It's null");
    } else {
      console.log("It's an object");
    }
  }
}

// Better approach for null checking
function betterCheck(value: string | object | null): void {
  if (value === null) {
    console.log("Null value");
    return;
  }
  
  if (typeof value === "string") {
    console.log("String:", value);
    return;
  }
  
  // TypeScript knows: value is object (and not null)
  console.log("Object:", value);
}
```

### 11.2.2 `in` Operator Checks

The `in` operator checks if a property exists on an object, which can be used for type narrowing with object types.

**Using `in` for Type Narrowing:**

```typescript
// Define distinct object types
interface Square {
  kind: "square";
  size: number;
}

interface Rectangle {
  kind: "rectangle";
  width: number;
  height: number;
}

interface Circle {
  kind: "circle";
  radius: number;
}

// Union type
type Shape = Square | Rectangle | Circle;

function getArea(shape: Shape): number {
  // Use 'in' to check for specific properties
  if ("size" in shape) {
    // TypeScript knows: shape is Square (only Square has 'size')
    return shape.size * shape.size;
  }
  
  if ("width" in shape && "height" in shape) {
    // TypeScript knows: shape is Rectangle
    return shape.width * shape.height;
  }
  
  // TypeScript knows: shape is Circle (only option left)
  return Math.PI * shape.radius * shape.radius;
}

// Better approach: discriminated unions (using 'kind' property)
function getAreaBetter(shape: Shape): number {
  // Check the discriminant property
  if (shape.kind === "square") {
    // TypeScript narrows to Square
    return shape.size * shape.size;
  }
  
  if (shape.kind === "rectangle") {
    // TypeScript narrows to Rectangle
    return shape.width * shape.height;
  }
  
  // TypeScript knows it's Circle
  return Math.PI * shape.radius * shape.radius;
}

// Using switch with discriminated unions
function describeShape(shape: Shape): string {
  switch (shape.kind) {
    case "square":
      return `Square with side ${shape.size}`;
    case "rectangle":
      return `Rectangle ${shape.width}x${shape.height}`;
    case "circle":
      return `Circle with radius ${shape.radius}`;
    default:
      // Exhaustiveness check
      const _exhaustiveCheck: never = shape;
      return _exhaustiveCheck;
  }
}
```

### 11.2.3 `instanceof` Checks

The `instanceof` operator checks if an object is an instance of a specific class, which is useful for type narrowing with class hierarchies.

**Using `instanceof` for Type Narrowing:**

```typescript
// Class hierarchy
class Animal {
  name: string;
  
  constructor(name: string) {
    this.name = name;
  }
  
  move(): void {
    console.log(`${this.name} is moving`);
  }
}

class Dog extends Animal {
  breed: string;
  
  constructor(name: string, breed: string) {
    super(name);
    this.breed = breed;
  }
  
  bark(): void {
    console.log("Woof! Woof!");
  }
  
  fetch(item: string): void {
    console.log(`${this.name} fetched the ${item}`);
  }
}

class Cat extends Animal {
  color: string;
  
  constructor(name: string, color: string) {
    super(name);
    this.color = color;
  }
  
  meow(): void {
    console.log("Meow!");
  }
  
  climb(): void {
    console.log(`${this.name} climbed a tree`);
  }
}

class Bird extends Animal {
  wingSpan: number;
  
  constructor(name: string, wingSpan: number) {
    super(name);
    this.wingSpan = wingSpan;
  }
  
  fly(): void {
    console.log(`${this.name} is flying with ${this.wingSpan}m wingspan`);
  }
}

// Function using instanceof for type narrowing
function interactWithAnimal(animal: Animal): void {
  // Common behavior available to all animals
  animal.move();
  
  // Type narrowing with instanceof
  if (animal instanceof Dog) {
    // TypeScript knows animal is Dog here
    animal.bark();      // OK - Dog specific
    animal.fetch("ball"); // OK - Dog specific
    console.log(`Breed: ${animal.breed}`); // OK - Dog specific
  } else if (animal instanceof Cat) {
    // TypeScript knows animal is Cat here
    animal.meow();      // OK - Cat specific
    animal.climb();     // OK - Cat specific
    console.log(`Color: ${animal.color}`); // OK - Cat specific
  } else if (animal instanceof Bird) {
    // TypeScript knows animal is Bird here
    animal.fly();       // OK - Bird specific
    console.log(`Wingspan: ${animal.wingSpan}m`); // OK - Bird specific
  } else {
    // TypeScript knows animal is just Animal (base class)
    console.log("Unknown animal type");
  }
}

// Usage
const animals: Animal[] = [
  new Dog("Rex", "German Shepherd"),
  new Cat("Whiskers", "Orange"),
  new Bird("Eagle", 2.5),
  new Animal("Generic Creature")
];

animals.forEach(animal => interactWithAnimal(animal));
```

**instanceof vs typeof:**

```typescript
// typeof is for primitives
function processPrimitive(value: string | number | boolean): void {
  if (typeof value === "string") {
    // value is string
  } else if (typeof value === "number") {
    // value is number
  } else if (typeof value === "boolean") {
    // value is boolean
  }
}

// instanceof is for classes/objects
class Cat {}
class Dog {}

function processObject(pet: Cat | Dog): void {
  if (pet instanceof Cat) {
    // pet is Cat
  } else {
    // pet is Dog
  }
}

// Note: instanceof doesn't work across realms (iframes) or with primitive wrappers
console.log("hello" instanceof String); // false (primitive)
console.log(new String("hello") instanceof String); // true (object)
```

### 11.2.4 Assignment Narrowing

Type narrowing also occurs through assignment, where TypeScript tracks the type of a variable based on what value was last assigned to it.

**Assignment-Based Narrowing:**

```typescript
// Type narrowing through assignment
let value = Math.random() > 0.5 ? "hello" : 100;
// TypeScript infers: value: string | number

if (typeof value === "string") {
  console.log(value.toUpperCase()); // OK - narrowed to string
} else {
  console.log(value.toFixed(2)); // OK - narrowed to number
}

// Control flow analysis
function processValue(value: string | number | boolean): void {
  // Initial type: string | number | boolean
  
  if (typeof value === "string") {
    // Narrowed to: string
    console.log(value.length);
    return; // Exit function
  }
  
  // After the if block and return:
  // Narrowed to: number | boolean
  
  if (typeof value === "number") {
    // Narrowed to: number
    console.log(value * 2);
  } else {
    // Narrowed to: boolean (only option left)
    console.log(value ? "yes" : "no");
  }
}

// Narrowing with equality checks
function checkStatus(status: "loading" | "success" | "error"): void {
  if (status === "loading") {
    console.log("Loading..."); // status is "loading"
  } else if (status === "success") {
    console.log("Done!"); // status is "success"
  } else {
    console.log("Failed!"); // status is "error" (exhausted)
  }
}
```

### 11.2.5 Control Flow Analysis

TypeScript's control flow analysis tracks how types change through conditional branches, loops, and other control structures.

**Advanced Control Flow:**

```typescript
// Type narrowing through control flow
function processData(data: string | string[] | null): void {
  // Check for null first (common pattern)
  if (data === null) {
    console.log("No data provided");
    return;
  }
  
  // After null check, data is string | string[]
  
  if (Array.isArray(data)) {
    // Narrowed to string[]
    console.log(`Processing ${data.length} items`);
    data.forEach(item => console.log(item));
  } else {
    // Narrowed to string
    console.log(`Processing single item: ${data}`);
  }
}

// Narrowing with custom type guards
interface Cat {
  type: "cat";
  meow(): void;
}

interface Dog {
  type: "dog";
  bark(): void;
}

function isCat(animal: Cat | Dog): animal is Cat {
  return animal.type === "cat";
}

function interact(animal: Cat | Dog): void {
  if (isCat(animal)) {
    animal.meow(); // TypeScript knows it's Cat
  } else {
    animal.bark(); // TypeScript knows it's Dog
  }
}

// Truthiness narrowing
function printLength(value: string | null | undefined): void {
  if (value) {
    // Narrowed to string (truthy)
    console.log(value.length);
  } else {
    console.log("No value");
  }
}

// Equality narrowing
function checkLength(x: string | number, y: string | boolean): void {
  if (x === y) {
    // Both must be string (only common type)
    console.log(x.toUpperCase()); // OK - x is string
    console.log(y.toUpperCase()); // OK - y is string
  } else {
    console.log(x); // string | number
    console.log(y); // string | boolean
  }
}
```

---

## 11.3 Discriminated Unions

Discriminated unions (also called tagged unions or algebraic data types) are a pattern for building type-safe unions using a common literal property (the discriminant) to distinguish between members.

### 11.3.1 Understanding Discriminated Unions

**The Discriminant Property:**

```typescript
// Each member has a 'kind' property (the discriminant)
interface Circle {
  kind: "circle";
  radius: number;
}

interface Square {
  kind: "square";
  side: number;
}

interface Rectangle {
  kind: "rectangle";
  width: number;
  height: number;
}

// Discriminated union
type Shape = Circle | Square | Rectangle;

// Type-safe function using discriminant
function getArea(shape: Shape): number {
  switch (shape.kind) {
    case "circle":
      // TypeScript knows shape is Circle here
      return Math.PI * shape.radius ** 2;
    case "square":
      // TypeScript knows shape is Square here
      return shape.side ** 2;
    case "rectangle":
      // TypeScript knows shape is Rectangle here
      return shape.width * shape.height;
    default:
      // Exhaustiveness check
      const _exhaustiveCheck: never = shape;
      return _exhaustiveCheck;
  }
}

// Usage
const circle: Circle = { kind: "circle", radius: 5 };
const square: Square = { kind: "square", side: 10 };
const rectangle: Rectangle = { kind: "rectangle", width: 5, height: 10 };

console.log(getArea(circle));     // 78.54...
console.log(getArea(square));     // 100
console.log(getArea(rectangle));  // 50
```

**Why Discriminated Unions:**

```
┌─────────────────────────────────────────────────────────────────────┐
│              Discriminated Unions vs Regular Unions                    │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│   Regular Union (Hard to distinguish):                              │
│   ┌──────────────────────────────────────────────────────────────┐ │
│   │ type Response = { status: number; data: any } |             │ │
│   │              { code: string; error: string };               │ │
│   │                                                                │ │
│   │ function handle(response: Response) {                        │ │
│   │   if ("data" in response) {                                  │ │
│   │     // Might be wrong if both have 'data'                    │ │
│   │   }                                                          │ │
│   │ }                                                            │ │
│   └──────────────────────────────────────────────────────────────┘ │
│                                                                     │
│   Discriminated Union (Clear and type-safe):                         │
│   ┌──────────────────────────────────────────────────────────────┐ │
│   │ type Success = { kind: "success"; data: any };                │ │
│   │ type Error = { kind: "error"; message: string };              │ │
│   │ type Response = Success | Error;                               │ │
│   │                                                                │ │
│   │ function handle(response: Response) {                          │ │
│   │   switch (response.kind) {                                     │ │
│   │     case "success":                                            │ │
│   │       // TypeScript knows this is Success                      │ │
│   │       console.log(response.data);                              │ │
│   │       break;                                                   │ │
│   │     case "error":                                              │ │
│   │       // TypeScript knows this is Error                        │ │
│   │       console.error(response.message);                         │ │
│   │       break;                                                   │ │
│   │   }                                                            │ │
│   │ }                                                            │ │
│   └──────────────────────────────────────────────────────────────┘ │
│                                                                     │
│   Benefits:                                                         │
│   • Compile-time exhaustiveness checking                            │
│   • Clear intent with 'kind' discriminator                          │
│   • IDE autocomplete knows exact shape in each branch               │
│   • Refactoring is safe - compiler catches missed cases            │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

### 11.3.2 The Discriminant Property

**Common Discriminant Patterns:**

```typescript
// HTTP Response discriminated union
type HttpResponse<T> =
  | { status: "loading" }
  | { status: "success"; data: T }
  | { status: "error"; error: string };

async function fetchData<T>(url: string): Promise<HttpResponse<T>> {
  try {
    // First return loading state
    const loading: HttpResponse<T> = { status: "loading" };
    
    const response = await fetch(url);
    
    if (!response.ok) {
      const errorResponse: HttpResponse<T> = {
        status: "error",
        error: `HTTP ${response.status}`
      };
      return errorResponse;
    }
    
    const data: T = await response.json();
    const successResponse: HttpResponse<T> = {
      status: "success",
      data
    };
    return successResponse;
    
  } catch (err) {
    const errorResponse: HttpResponse<T> = {
      status: "error",
      error: err instanceof Error ? err.message : "Unknown error"
    };
    return errorResponse;
  }
}

// UI Component state
type ComponentState =
  | { type: "idle" }
  | { type: "loading"; progress: number }
  | { type: "loaded"; data: unknown }
  | { type: "error"; message: string; retryable: boolean };

function renderComponent(state: ComponentState): string {
  switch (state.type) {
    case "idle":
      return "Waiting for input...";
      
    case "loading":
      return `Loading... ${state.progress}%`;
      
    case "loaded":
      return `Data loaded: ${JSON.stringify(state.data)}`;
      
    case "error":
      return `Error: ${state.message}${
        state.retryable ? " (Click to retry)" : ""
      }`;
  }
}
```

### 11.3.3 Exhaustiveness Checking with `never`

**Ensuring All Cases Are Handled:**

```typescript
// Exhaustiveness checking ensures all union members are handled
type Shape =
  | { kind: "circle"; radius: number }
  | { kind: "square"; side: number }
  | { kind: "triangle"; base: number; height: number }
  | { kind: "rectangle"; width: number; height: number };

function getArea(shape: Shape): number {
  switch (shape.kind) {
    case "circle":
      return Math.PI * shape.radius ** 2;
    case "square":
      return shape.side ** 2;
    case "triangle":
      return (shape.base * shape.height) / 2;
    // Forgot rectangle!
    default:
      // TypeScript error: Type 'rectangle' is not assignable to type 'never'
      const _exhaustiveCheck: never = shape;
      throw new Error(`Unhandled shape: ${_exhaustiveCheck}`);
  }
}

// Helper function for exhaustiveness
function assertNever(value: never): never {
  throw new Error(`Unexpected value: ${value}`);
}

// Better implementation
function getAreaSafe(shape: Shape): number {
  switch (shape.kind) {
    case "circle":
      return Math.PI * shape.radius ** 2;
    case "square":
      return shape.side ** 2;
    case "triangle":
      return (shape.base * shape.height) / 2;
    case "rectangle":
      return shape.width * shape.height;
    default:
      return assertNever(shape); // Compile-time check
  }
}

// Adding a new shape requires updating all switch statements
// TypeScript will catch missing cases at compile time
```

---

## 11.4 Intersection Types

Intersection types combine multiple types into one, requiring a value to satisfy all types simultaneously (AND relationship, as opposed to union's OR).

### 11.4.1 Defining Intersection Types

**Basic Intersection Syntax:**

```typescript
// Intersection type: must have ALL properties from both types
type HasName = { name: string };
type HasAge = { age: number };

// Person must have both name AND age
type Person = HasName & HasAge;

const person: Person = {
  name: "John",
  age: 30
};

// Missing either property causes error
// const incomplete: Person = { name: "John" }; 
// ❌ Error: Property 'age' is missing

// Intersection with multiple types
type Employee = HasName & HasAge & {
  employeeId: string;
  department: string;
};

const employee: Employee = {
  name: "Jane",
  age: 28,
  employeeId: "EMP001",
  department: "Engineering"
};

// Intersection with interfaces
interface Drawable {
  draw(): void;
}

interface Movable {
  move(x: number, y: number): void;
}

// Sprite has all methods from both interfaces
type Sprite = Drawable & Movable;

class GameSprite implements Sprite {
  draw(): void {
    console.log("Drawing sprite");
  }
  
  move(x: number, y: number): void {
    console.log(`Moving to (${x}, ${y})`);
  }
}
```

**Intersection Type Visualization:**

```
┌─────────────────────────────────────────────────────────────────────┐
│                    Intersection Type (A & B)                         │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│   Type A: { name: string; age: number }                             │
│   Type B: { age: number; email: string }                            │
│                                                                     │
│   A & B: { name: string; age: number; email: string }              │
│                                                                     │
│   ┌──────────────────────────────────────────────────────────────┐ │
│   │                    Venn Diagram                               │ │
│   │                                                                │ │
│   │         ┌─────────┐      ┌─────────┐                        │ │
│   │         │  Type A │      │  Type B │                        │ │
│   │         │         │      │         │                        │ │
│   │         │  name   │      │  email  │                        │ │
│   │         │  age    │  &   │  age    │  =  name, age, email   │ │
│   │         │         │      │         │                        │ │
│   │         └─────────┘      └─────────┘                        │ │
│   │                                                                │ │
│   └──────────────────────────────────────────────────────────────┘ │
│                                                                     │
│   Key Points:                                                       │
│   • All properties from both types are required                     │
│   • If types share properties, they must be compatible              │
│   • Creates a new type that satisfies both                          │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

### 11.4.2 Combining Types

**Intersection with Conflicting Types:**

```typescript
// When properties overlap, they must be compatible
type A = { x: string };
type B = { x: string; y: number };

// C has x: string and y: number
type C = A & B; // { x: string; y: number }

// Incompatible types cause never
type D = { x: string };
type E = { x: number };

type F = D & E;
// x becomes string & number which is never (no value can be both)
// const f: F = { x: "test" }; // Error: Type 'string' is not assignable to type 'never'

// Intersection with functions
type Fn1 = (x: string) => void;
type Fn2 = (x: number) => void;

// Intersection of functions accepts both types (overloaded)
type CombinedFn = Fn1 & Fn2;

const fn: CombinedFn = (x: string | number) => {
  console.log(x);
};

fn("hello"); // OK
fn(42);      // OK
// fn(true);  // Error

// Intersection with index signatures
type WithStringIndex = { [key: string]: string };
type WithNumberIndex = { [key: number]: number };

// Combined must satisfy both
type CombinedIndex = WithStringIndex & WithNumberIndex;

// This is tricky because string index and number index conflict
// const c: CombinedIndex = {
//   "0": "string",  // Must be string for string index, but number for number index
//   0: 123          // Must be number for number index
// };
```

### 11.4.3 Intersection with Primitive Types

**Primitive Intersections:**

```typescript
// Intersection of primitives becomes never
type StringAndNumber = string & number; // never
// No value can be both a string and a number simultaneously

// Intersection with object wrappers
type StringObject = String & { custom: boolean };
// Results in type with both String methods and custom property

// Practical use: Branding/Opaque types
type UserId = string & { __brand: "UserId" };
type OrderId = string & { __brand: "OrderId" };

function createUserId(id: string): UserId {
  return id as UserId;
}

function createOrderId(id: string): OrderId {
  return id as OrderId;
}

const userId = createUserId("user-123");
const orderId = createOrderId("order-456");

function getUser(id: UserId): void {
  console.log(`Getting user ${id}`);
}

function getOrder(id: OrderId): void {
  console.log(`Getting order ${id}`);
}

getUser(userId);     // ✅ OK
getOrder(orderId);   // ✅ OK
// getUser(orderId); // ❌ Error: OrderId is not assignable to UserId
// getOrder(userId); // ❌ Error: UserId is not assignable to OrderId

// Even though both are strings at runtime, TypeScript treats them as distinct types
```

---

## 11.5 Union vs Intersection: When to Use Each

Understanding when to use union types versus intersection types is crucial for effective TypeScript design.

**Comparison and Use Cases:**

```typescript
// UNION (|) - "One of these" (OR)
// Use when: Value could be one of several types

// Example: API response that could be success or error
type ApiResponse<T> = 
  | { status: "success"; data: T }
  | { status: "error"; error: string }
  | { status: "loading" };

// Function parameter accepting multiple types
function formatInput(input: string | number | Date): string {
  if (typeof input === "string") return input;
  if (typeof input === "number") return input.toString();
  return input.toISOString();
}

// INTERSECTION (&) - "All of these" (AND)
// Use when: Value must satisfy all types simultaneously

// Example: Object that is both a User and has Admin privileges
type User = { name: string; email: string };
type Admin = { permissions: string[]; level: number };
type AdminUser = User & Admin; // Has all properties of both

const admin: AdminUser = {
  name: "John",
  email: "john@example.com",
  permissions: ["read", "write", "delete"],
  level: 5
};

// Extending interfaces (similar to intersection)
interface Serializable {
  serialize(): string;
}

interface Timestamped {
  createdAt: Date;
}

// Class implementing intersection of interfaces
class Document implements Serializable & Timestamped {
  constructor(
    public content: string,
    public createdAt: Date = new Date()
  ) {}
  
  serialize(): string {
    return JSON.stringify({
      content: this.content,
      createdAt: this.createdAt.toISOString()
    });
  }
}

// Decision Tree:
// 
// Do you need the value to be ONE OF several types?
// → Use UNION (|)
//    Example: string | number
//    Use case: "This parameter accepts either strings or numbers"
//
// Do you need the value to be ALL OF several types?
// → Use INTERSECTION (&)
//    Example: Person & Serializable
//    Use case: "This object must be both a Person AND Serializable"
//
// Common confusion:
// type A = { x: string }
// type B = { y: number }
// 
// A | B means: { x: string } OR { y: number } (one or the other)
// A & B means: { x: string, y: number } (both together)
```

---

## 11.6 Chapter Summary and Exercises

### Chapter Summary

In this chapter, we covered union and intersection types comprehensively:

**Key Takeaways:**

1. **Union Types (`|`)**:
   - Values can be one of several types
   - Used for flexible parameters and return types
   - Require type narrowing to access specific properties

2. **Type Narrowing**:
   - `typeof` for primitive types
   - `instanceof` for class instances
   - `in` operator for object properties
   - Custom type guards using `is`
   - Assignment and control flow analysis

3. **Discriminated Unions**:
   - Use a common literal property (discriminant) to distinguish types
   - Enable exhaustive checking with switch statements
   - Provide excellent IDE support and type safety
   - Pattern for state management (Redux, etc.)

4. **Intersection Types (`&`)**:
   - Values must satisfy all types simultaneously
   - Combine multiple object types
   - Used for mixins and extending types
   - Can create branded/opaque types

5. **Nullable Types**:
   - `T | null | undefined` for optional values
   - Strict null checks require handling null cases
   - Non-null assertion operator (`!`) for when you're certain

6. **Exhaustiveness Checking**:
   - Use `never` type to ensure all cases are handled
   - Compiler error if union member is not handled
   - Essential for maintainable discriminated unions

### Practical Exercises

**Exercise 1: Union Types**

Create a flexible formatting system:

```typescript
// 1. Create a union type Value = string | number | boolean | Date
// 2. Write a function format(value: Value, format?: string): string that:
//    - For strings: returns uppercase or applies format as regex replace
//    - For numbers: formats with given decimal places or as currency
//    - For booleans: returns "Yes"/"No" or "Active"/"Inactive"
//    - For Date: formats as ISO, locale, or custom format string
// 3. Use type narrowing with typeof and instanceof
// 4. Handle edge cases (null, undefined, NaN)
```

**Exercise 2: Discriminated Unions**

Build a state machine for a payment process:

```typescript
// 1. Create states for a payment flow:
//    - Idle: { status: "idle" }
//    - Validating: { status: "validating"; amount: number; currency: string }
//    - Processing: { status: "processing"; transactionId: string; startTime: Date }
//    - Success: { status: "success"; transactionId: string; completedAt: Date; receipt: string }
//    - Failed: { status: "failed"; error: string; code: number; canRetry: boolean }
// 2. Write a function processPayment(state: PaymentState) that handles each state
// 3. Implement transitions between states
// 4. Add exhaustiveness checking to ensure all states are handled
// 5. Create a type guard isTerminalState(state): state is Success | Failed
```

**Exercise 3: Type Guards**

Implement type-safe API response handling:

```typescript
// 1. Define types:
//    - User = { id: number; name: string; email: string }
//    - Product = { id: number; title: string; price: number }
//    - Order = { id: number; userId: number; items: number[]; total: number }
// 2. Create a union type ApiEntity = User | Product | Order
// 3. Write type guards:
//    - isUser(entity): entity is User
//    - isProduct(entity): entity is Product
//    - isOrder(entity): entity is Order
// 4. Create a function processEntity(entity: ApiEntity) that:
//    - Uses type guards to handle each type appropriately
//    - Returns different summaries for each type
// 5. Handle an array of mixed entities using filter with type guards
```

**Exercise 4: Intersection Types**

Create composable behaviors:

```typescript
// 1. Define base types:
//    - Timestamped = { createdAt: Date; updatedAt: Date; update(): void }
//    - Versioned = { version: number; incrementVersion(): void }
//    - Serializable = { serialize(): string; toJSON(): object }
//    - Validatable = { validate(): boolean; errors: string[] }
// 2. Create intersection types combining these:
//    - Document = Timestamped & Versioned & Serializable
//    - FormData = Validatable & Timestamped
// 3. Implement classes that satisfy these intersections
// 4. Write utility functions that accept intersection types
// 5. Demonstrate how intersection types enforce all requirements
```

**Exercise 5: Nullable Types and Exhaustiveness**

Build a safe parser:

```typescript
// 1. Create a Result<T> type using discriminated union:
//    - Success = { kind: "success"; value: T }
//    - Failure = { kind: "failure"; error: string; code: number }
// 2. Write a function parseJSON<T>(json: string): Result<T> that:
//    - Returns Success with parsed data if valid
//    - Returns Failure with error details if invalid
// 3. Create a function unwrapOrDefault<T>(result: Result<T>, defaultValue: T): T
// 4. Implement pattern matching helper that handles all cases with exhaustiveness
// 5. Use strict null checks to ensure safety
```

### Additional Resources

- **TypeScript Handbook - Union Types**: https://www.typescriptlang.org/docs/handbook/2/everyday-types.html#union-types
- **TypeScript Handbook - Narrowing**: https://www.typescriptlang.org/docs/handbook/2/narrowing.html
- **Discriminated Unions in TypeScript**: https://www.typescriptlang.org/docs/handbook/2/narrowing.html#discriminated-unions
- **Exhaustiveness Checking**: https://www.typescriptlang.org/docs/handbook/2/narrowing.html#exhaustiveness-checking

---

## Coming Up Next: Chapter 12 - Type Aliases

In the next chapter, we will explore:

- Creating type aliases with the `type` keyword
- Aliasing primitive types, object types, and union types
- Type aliases vs Interfaces - key differences and when to use each
- Recursive type aliases
- Type aliases with generics
- Template literal types
- Naming conventions and best practices for type aliases

Type aliases provide a powerful way to create reusable, semantic names for types, making your code more readable and maintainable while offering flexibility that complements TypeScript's interface system.

---